In [14]:
import requests
import pandas as pd
import time
import re

# 1. 파일 읽기
filename = "서울특별시 성북구_가로 휴지통 위치 및 개수_20210928.csv"
try:
    df = pd.read_csv(filename, encoding='cp949')
except:
    df = pd.read_csv(filename, encoding='utf-8')

# 2. 카카오 API 함수
def get_location(query, api_key):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": query}
    
    try:
        response = requests.get(url, params=params, headers=headers)
        if response.status_code == 200:
            documents = response.json().get('documents')
            if documents:
                # 검색 결과 중 첫 번째 결과의 정보 반환
                place = documents[0]
                return place['x'], place['y'], place['place_name'], place['address_name']
        return None, None, None, None
    except Exception as e:
        return None, None, None, None

# 3. 검색 로직 실행
api_key = "29cf96c3bebe9f8caec569384f45f2b4"
locations = []

print("좌표 변환을 시작합니다...")

for idx, row in df.iterrows():
    raw_text = row['설치장소']
    lon, lat, found_name, found_addr = None, None, None, None
    method = ""
    search_query = ""

    # [1단계] 버스정류장 ID (08-xxx) 추출 및 검색
    bus_id_match = re.search(r'\d{2}-\d{3,4}', raw_text)
    if bus_id_match:
        search_query = bus_id_match.group()
        lon, lat, found_name, found_addr = get_location(search_query, api_key)
        method = "버스ID"

    # [2단계] 실패 시, 주소 부분(점 앞부분)으로 검색
    if lat is None:
        if '.' in raw_text:
            addr_part = raw_text.split('.')[0].strip()
            # "아리랑로8" -> "아리랑로 8" (숫자 앞 띄어쓰기)
            addr_part = re.sub(r'([가-힣])(\d)', r'\1 \2', addr_part)
            search_query = f"성북구 {addr_part}"
            lon, lat, found_name, found_addr = get_location(search_query, api_key)
            method = "주소검색"
    
    # [3단계] 실패 시, 장소명 키워드(점 뒷부분)로 검색 (사용자님 아이디어!)
    if lat is None:
        if '.' in raw_text:
            # 점 뒤에 있는 내용 가져오기 (예: "성신여대입구역 버스정류장 (08-270)")
            keyword_part = raw_text.split('.')[1].strip()
            # 괄호 뒤에 있는 불필요한 정보 제거 (예: "(08-270). 농협" -> "성신여대입구역 버스정류장")
            keyword_part = keyword_part.split('(')[0].strip()
            
            if keyword_part:
                search_query = f"성북구 {keyword_part}"
                lon, lat, found_name, found_addr = get_location(search_query, api_key)
                method = "장소명검색"

    # [4단계] 최후의 수단: 전체 텍스트 검색
    if lat is None:
        search_query = raw_text
        lon, lat, found_name, found_addr = get_location(search_query, api_key)
        method = "전체검색"

    # 결과 기록
    if lat:
        print(f"[성공] ({method}) {search_query} -> {found_name}")
    else:
        print(f"[실패] {raw_text}")

    locations.append({
        '설치장소_원본': raw_text,
        '성공방법': method,
        '사용검색어': search_query,
        '찾은장소명': found_name,
        '찾은주소': found_addr,
        '위도': lat,
        '경도': lon
    })
    
    time.sleep(0.3)

# 4. 파일 저장
result_df = pd.DataFrame(locations)
final_df = pd.concat([df, result_df[['위도', '경도', '성공방법', '찾은주소']]], axis=1)
final_df.to_csv("성북구_가로휴지통_좌표_최종완성.csv", index=False, encoding='cp949')

print("\n모든 작업이 완료되었습니다!")

좌표 변환을 시작합니다...
[실패] 동소문로2길 입구. 한성대입구역 분수광장
[실패] 한성대입구 중앙버스정류장 (혜화역 - 길음역) 08-010
[실패] 한성대입구 중앙버스정류장 (혜화역 - 길음역) 08-010
[실패] 한성대입구 중앙버스정류장 (혜화역 - 길음역) 08-010
[실패] 한성대입구 중앙버스정류장 (길음역 - 혜화역) 08-009
[실패] 한성대입구 중앙버스정류장 (길음역 - 혜화역) 08-009
[실패] 한성대입구 중앙버스정류장 (길음역 - 혜화역) 08-009
[실패] 한성대입구 중앙버스정류장 (길음역 - 혜화역) 08-009
[실패] 성북로5. 한성대입구역 5번출구
[실패] 보문로168. 성북구청 바람마당
[실패] 성신여대입구 중앙버스정류장 (미아리고개 - 한성대입구) 08-007


KeyboardInterrupt: 

In [20]:
import requests
import pandas as pd
import time
import re

# 1. 파일 읽기
filename = "서울특별시 성북구_가로 휴지통 위치 및 개수_20210928.csv"
try:
    df = pd.read_csv(filename, encoding='cp949')
except:
    df = pd.read_csv(filename, encoding='utf-8')

# 2. 카카오 API 함수 (JSON 구조 반영)
def get_location(query, api_key):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": query, "size": 1} # 가장 정확한 1개만 요청
    
    try:
        response = requests.get(url, params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            documents = data.get('documents')
            
            # 검색 결과가 있을 때만 처리
            if documents:
                place = documents[0] # 첫 번째 결과 (가장 정확도 높음)
                
                # 필요한 정보 추출 (제공해주신 JSON 구조 참고)
                lat = place.get('y') # 위도
                lon = place.get('x') # 경도
                name = place.get('place_name') # 장소명
                addr = place.get('road_address_name') or place.get('address_name') # 도로명 없으면 지번주소
                
                return lon, lat, name, addr
                
        return None, None, None, None
    except Exception as e:
        print(f"API Error: {e}")
        return None, None, None, None

# 3. 메인 로직 실행
api_key = "1a995967389e24f4751df5ceba9beb29" # ★★★ API 키 입력 필수 ★★★
locations = []

print("=== 좌표 변환 시작 ===")

for idx, row in df.iterrows():
    raw_text = row['설치장소']
    lon, lat, found_name, found_addr = None, None, None, None
    method = ""
    search_query = ""

    # --- [전략 1] 버스정류장 ID (08-xxx) ---
    bus_id_match = re.search(r'\d{2}-\d{3,4}', raw_text)
    if bus_id_match:
        search_query = bus_id_match.group() # "성북구" 붙이지 않음 (ID는 고유함)
        lon, lat, found_name, found_addr = get_location(search_query, api_key)
        method = "버스ID"

    # --- [전략 2] 주소 검색 (도로명 + 번지) ---
    if lat is None:
        if '.' in raw_text:
            addr_part = raw_text.split('.')[0].strip()
            # 띄어쓰기 보정 ("아리랑로8" -> "아리랑로 8")
            addr_part = re.sub(r'([가-힣])(\d)', r'\1 \2', addr_part)
            
            # 주소 형식이 맞는지 간단 체크 (숫자가 포함되어 있어야 주소일 확률 높음)
            if re.search(r'\d', addr_part):
                search_query = f"성북구 {addr_part}"
                lon, lat, found_name, found_addr = get_location(search_query, api_key)
                method = "주소검색"

    # --- [전략 3] 키워드/장소명 검색 (사용자 제안) ---
    if lat is None:
        # 1. 괄호 앞의 텍스트 추출 (예: "성신여대입구역 버스정류장 (08-270)" -> "성신여대입구역 버스정류장")
        keyword_candidate = raw_text.split('(')[0].strip()
        
        # 2. 점(.) 뒤에 있는 경우, 뒷부분 사용 (예: "아리랑로8. 성신여대..." -> "성신여대...")
        if '.' in keyword_candidate:
            keyword_candidate = keyword_candidate.split('.')[1].strip()
            
        if len(keyword_candidate) > 1: # 한 글자는 검색 무의미
            search_query = f"성북구 {keyword_candidate}"
            lon, lat, found_name, found_addr = get_location(search_query, api_key)
            method = "키워드검색"

    # --- [전략 4] 최후의 수단 (원본 전체) ---
    if lat is None:
        search_query = raw_text
        lon, lat, found_name, found_addr = get_location(f"성북구 {search_query}", api_key)
        method = "전체검색"

    # 결과 출력 및 저장
    if lat:
        print(f"[성공] {raw_text[:15]}.. -> {found_name} ({method})")
    else:
        print(f"[실패] {raw_text[:15]}..")

    locations.append({
        '설치장소_원본': raw_text,
        '성공방법': method,
        '사용검색어': search_query,
        '찾은장소명': found_name,
        '찾은주소': found_addr,
        '위도': lat,
        '경도': lon
    })
    
    time.sleep(0.05) # API 호출 간격 조절

# 4. CSV 저장
result_df = pd.DataFrame(locations)
final_df = pd.concat([df, result_df[['위도', '경도', '성공방법', '찾은장소명']]], axis=1)
final_df.to_csv("성북구_가로휴지통_좌표_완성본.csv", index=False, encoding='cp949')

print("\n변환 완료! '성북구_가로휴지통_좌표_완성본.csv' 파일이 생성되었습니다.")

=== 좌표 변환 시작 ===
[성공] 동소문로2길 입구. 한성대입.. -> 성북천 분수광장 (키워드검색)
[실패] 한성대입구 중앙버스정류장 (..
[실패] 한성대입구 중앙버스정류장 (..
[실패] 한성대입구 중앙버스정류장 (..
[실패] 한성대입구 중앙버스정류장 (..
[실패] 한성대입구 중앙버스정류장 (..
[실패] 한성대입구 중앙버스정류장 (..
[실패] 한성대입구 중앙버스정류장 (..
[성공] 성북로5. 한성대입구역 5번.. -> 한성대입구역 4호선 5번출구 (키워드검색)
[성공] 보문로168. 성북구청 바람.. -> 성북구청 (주소검색)
[실패] 성신여대입구 중앙버스정류장 ..
[실패] 성신여대입구 중앙버스정류장 ..
[실패] 성신여대입구 중앙버스정류장 ..
[실패] 성신여대입구 중앙버스정류장 ..
[실패] 성신여대입구 중앙버스정류장 ..
[실패] 성신여대입구 중앙버스정류장 ..
[성공] 아리랑로8.  성신여대입구역.. -> 아리랑힐호텔 동대문 (주소검색)
[성공] 아리랑로7-1. 성신여대입구.. -> 성신필약국 (주소검색)
[실패] 정릉로44길 입구. 돈암삼성..
[실패] 정릉로394. 돈암1동주민센..
[실패] 미아리고개 중앙버스정류장 (..
[실패] 미아리고개 중앙버스정류장 (..
[실패] 미아리고개 중앙버스정류장 (..
[실패] 미아리고개 중앙버스정류장 (..
[성공] 성북로165. 서울다원학교 .. -> 신원그린빌라 (주소검색)
[성공] 정릉로31길 30. 숭덕초교.. -> 정릉1치안센터 (주소검색)
[실패] 정릉로 282. 숭덕초교 버..
[실패] 정릉로 26길 1. 정릉2동..
[실패] 보국문로6 맞은 편 광장. ..
[성공] 정릉로161. 고려사대부고 .. -> 고려대학교 사범대학 부속고등학교 (주소검색)
[실패] 정릉로9길 입구. 청덕초교 ..
[실패] 정릉로77 국민대학교 맞은 ..
[성공] 정릉로77. 국민대학교 버스.. -> 국민대학교 (주소검색)
[성공] 정릉로77. 국민대앞 버스정.. -> 국민대학교 (주소검색)
[실패] 정릉

In [64]:
import requests
headers = {"Authorization": "KakaoAK 1a995967389e24f4751df5ceba9beb29"}

r = requests.get(
    "https://dapi.kakao.com/v2/local/search/keyword.json",
    params={"query": "돈암삼성아파트"},
    headers=headers
)

print(r.status_code)
print(r.text)

200
{"documents":[{"address_name":"서울 성북구 돈암동 15-1","category_group_code":"","category_group_name":"","category_name":"부동산 \u003e 주거시설 \u003e 아파트","distance":"","id":"11311640","phone":"","place_name":"돈암삼성아파트","place_url":"http://place.map.kakao.com/11311640","road_address_name":"서울 성북구 동소문로34길 24","x":"127.02331371539344","y":"37.59903074165889"},{"address_name":"서울 성북구 돈암동 15-1","category_group_code":"","category_group_name":"","category_name":"서비스,산업 \u003e 관리,운영 \u003e 건물관리사무소","distance":"","id":"9795105","phone":"02-921-4844","place_name":"돈암삼성아파트 관리사무소","place_url":"http://place.map.kakao.com/9795105","road_address_name":"서울 성북구 동소문로34길 24","x":"127.022946743587","y":"37.5987821400672"},{"address_name":"서울 성북구 돈암동 15-1","category_group_code":"","category_group_name":"","category_name":"가정,생활 \u003e 상가,아케이드 \u003e 아파트상가","distance":"","id":"14971868","phone":"","place_name":"돈암삼성아파트 B상가","place_url":"http://place.map.kakao.com/14971868","road_address_name":"서울 성북구 동소문로34길 24","x

In [66]:
import pandas as pd

file_path = "성북구_가로휴지통_좌표_완성본.csv"

# Load the file, trying common encodings
try:
    df = pd.read_csv(file_path, encoding='cp949')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='utf-8')

# Display column names and first few rows to map A, B, C, F, G, I columns
print("Columns:", df.columns.tolist())
print(df.head())
print(df.iloc[0])

# 1. Drop rows where '위도' (Column F, index 5) is NaN
df_clean = df.dropna(subset=['위도']).copy()

# 2. Drop duplicate rows based on '위도' and '경도' (Column F and G)
# "f,g열 동일한 행" interpreted as duplicate coordinates
df_clean = df_clean.drop_duplicates(subset=['위도', '경도'])

# 3. Rearrange columns
# I (찾은장소명) -> A position
# F (위도) -> B position
# G (경도) -> C position
# We will create a new dataframe with these as the leading columns.
# We'll include the original '설치장소' as well just in case, but prioritize the requested mapping.

cols = df_clean.columns.tolist()
# Map: I='찾은장소명', F='위도', G='경도'
selected_cols = ['찾은장소명', '위도', '경도']

# Optional: Fill '찾은장소명' with '설치장소' if it's missing, for better readability
df_clean['찾은장소명'] = df_clean['찾은장소명'].fillna(df_clean['설치장소'])

# Create final dataframe with requested order
final_df = df_clean[selected_cols].copy()

# Display info
print(final_df.info())
print(final_df.head())

# Save
final_df.to_csv("성북구_가로휴지통_전처리완료.csv", index=False, encoding='cp949')

Columns: ['구분', '설치장소', '관리기관', '관리기관 전화번호', '데이터 기준일자', '위도', '경도', '성공방법', '찾은장소명']
   구분                              설치장소        관리기관     관리기관 전화번호   데이터 기준일자  \
0   1            동소문로2길 입구. 한성대입구역 분수광장  서울특별시 성북구청  02-2241-4418  2021.9.28   
1   2  한성대입구 중앙버스정류장 (혜화역 - 길음역) 08-010  서울특별시 성북구청  02-2241-4418  2021.9.28   
2   3  한성대입구 중앙버스정류장 (혜화역 - 길음역) 08-010  서울특별시 성북구청  02-2241-4418  2021.9.28   
3   4  한성대입구 중앙버스정류장 (혜화역 - 길음역) 08-010  서울특별시 성북구청  02-2241-4418  2021.9.28   
4   5  한성대입구 중앙버스정류장 (길음역 - 혜화역) 08-009  서울특별시 성북구청  02-2241-4418  2021.9.28   

         위도         경도   성공방법     찾은장소명  
0  37.58817  127.00673  키워드검색  성북천 분수광장  
1       NaN        NaN   전체검색       NaN  
2       NaN        NaN   전체검색       NaN  
3       NaN        NaN   전체검색       NaN  
4       NaN        NaN   전체검색       NaN  
구분                                1
설치장소         동소문로2길 입구. 한성대입구역 분수광장
관리기관                     서울특별시 성북구청
관리기관 전화번호              02-2241-4418
데이터 기준일자                  2021.9.28
위

In [77]:
import pandas as pd
import folium

# 1. 데이터 불러오기
file_path = "성북구_가로휴지통_전처리완료.csv"
try:
    df = pd.read_csv(file_path, encoding='cp949')
except:
    df = pd.read_csv(file_path, encoding='utf-8')

# 2. 지도의 중심 잡기
center_lat = df['위도'].mean()
center_lon = df['경도'].mean()

# 3. 지도 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# 4. 마커 추가 (Cluster 기능 없이 바로 지도 m에 추가)
for idx, row in df.iterrows():
    # 마커 클릭 시 나올 팝업 텍스트
    popup_text = folium.Popup(str(row['찾은장소명']), max_width=300)
    
    # 마커 생성 및 지도에 추가
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=popup_text,
        # 아이콘 모양 설정 (기본 파란색 핀)
        icon=folium.Icon(color='blue', icon='trash', prefix='fa') 
    ).add_to(m) # 여기에 marker_cluster가 아니라 m을 씁니다.

# 5. HTML 파일로 저장
m.save("성북구_휴지통_지도_전체보기.html")

print("완료! '성북구_휴지통_지도_전체보기.html' 파일이 생성되었습니다.")

완료! '성북구_휴지통_지도_전체보기.html' 파일이 생성되었습니다.


In [79]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import haversine_distances

# 1. 전처리된 파일 불러오기
file_path = "성북구_가로휴지통_전처리완료.csv"
try:
    df = pd.read_csv(file_path, encoding='cp949')
except:
    df = pd.read_csv(file_path, encoding='utf-8')

# 2. 거리 계산 (Haversine Formula)
# 위도/경도를 라디안으로 변환
df['lat_rad'] = np.radians(df['위도'])
df['lon_rad'] = np.radians(df['경도'])
coords = df[['lat_rad', 'lon_rad']].values

# 거리 행렬 계산 (결과는 라디안)
dist_matrix_rad = haversine_distances(coords)

# 미터 단위 변환 (지구 반지름 약 6371km)
dist_matrix_m = dist_matrix_rad * 6371000

# 3. 데이터프레임 만들기 (인덱스와 컬럼을 장소명으로 설정)
dist_df = pd.DataFrame(
    dist_matrix_m, 
    index=df['찾은장소명'], 
    columns=df['찾은장소명']
)

# 보기 좋게 정수형으로 반올림
dist_df = dist_df.round(0).astype(int)

# 4. CSV 저장
output_filename = "성북구_휴지통_직선거리행렬.csv"
dist_df.to_csv(output_filename, encoding='cp949') # 엑셀 호환용 인코딩

print(f"파일 생성 완료: {output_filename}")



파일 생성 완료: 성북구_휴지통_직선거리행렬.csv
